# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [18]:
import numpy as np
import pandas as pd
import requests
import math
import xlsxwriter
from scipy import stats

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [19]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [20]:
symbol ='AAPL'
api_url="https://cloud.iexapis.com/stable"
later_url=f"/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}"
data=requests.get(api_url+later_url).json()
print(data)

{'companyName': 'Apple Inc', 'marketcap': 2808447038460, 'week52high': 171.58, 'week52low': 115.67, 'week52highSplitAdjustOnly': 171.58, 'week52lowSplitAdjustOnly': 116.21, 'week52change': 0.38475091390486327, 'sharesOutstanding': 16406397000, 'float': 0, 'avg10Volume': 128269129, 'avg30Volume': 97350766, 'day200MovingAvg': 144.93, 'day50MovingAvg': 154.71, 'employees': 147000, 'ttmEPS': 11.23, 'ttmDividendRate': 0.8630784880892401, 'dividendYield': 0.005041935320067999, 'nextDividendDate': '', 'exDividendDate': '2021-11-05', 'nextEarningsDate': '2022-01-26', 'peRatio': 14.831258124524716, 'beta': 1.3267519773221672, 'maxChangePercent': 64.89421818461776, 'year5ChangePercent': 5.551692463142424, 'year2ChangePercent': 1.5669209394948638, 'year1ChangePercent': 0.391800103746754, 'ytdChangePercent': 0.2980277076366613, 'month6ChangePercent': 0.3636743847014823, 'month3ChangePercent': 0.09407036614734854, 'month1ChangePercent': 0.1315441565309361, 'day30ChangePercent': 0.1315441565309361, 

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [21]:
print(data['year1ChangePercent'])
print(data['month6ChangePercent'])
print(data['month3ChangePercent'])
print(data['month1ChangePercent'])

0.391800103746754
0.3636743847014823
0.09407036614734854
0.1315441565309361


## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [38]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])
print(symbol_strings)
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

['A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA', 'CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD', 'GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [36]:
resultant_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_url = f"/stock/market/batch?symbols={symbol_string}&types=quote,stats&token={IEX_CLOUD_API_TOKEN}"
    #print(api_url+batch_api_url)
    data = requests.get(api_url+batch_api_url).json()
    
    for symbol in symbol_string.split(','):
        resultant_dataframe=resultant_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['stats']['year1ChangePercent'],
                    "N/A"
                ],index=my_columns
            ),ignore_index=True
        )
    
resultant_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,AAPL,174.27,0.3918,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [24]:
resultant_dataframe.sort_values('One-Year Price Return',ascending=False,inplace=True)
resultant_dataframe=resultant_dataframe[:50]
resultant_dataframe.reset_index(inplace=True)
resultant_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,79.920,2.281287,N/A
1,148,DVN,44.955,1.904802,N/A
2,175,FANG,114.480,1.585831,N/A
3,299,MCHP,87.080,1.479823,N/A
4,195,FTNT,320.530,1.476889,N/A
5,317,MRO,16.250,1.46531,N/A
6,345,NVDA,317.900,1.385236,N/A
7,174,F,19.905,1.175761,N/A
8,344,NUE,113.240,1.08169,N/A
9,251,IT,319.490,1.031696,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [25]:
def portfolio_input():
    global portfolio_size
    
    portfolio_size=input("Enter the value you want to invest")
    try:
        val=float(portfolio_size)
    except ValueError:
        print("Please Enter an Integer/Numeric Value")
        
portfolio_input()
print(f"Portfolio Size: {portfolio_size}")

Enter the value you want to invest 100000


Portfolio Size: 100000


In [26]:
position_size=float(portfolio_size)/len(resultant_dataframe.index)
print(f"Position Size {position_size}")

for i in range(len(resultant_dataframe.index)):
    resultant_dataframe.loc[i,"Number of Shares to Buy"] = math.floor(position_size/resultant_dataframe.loc[i,"Price"])
    
resultant_dataframe

Position Size 2000.0


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,79.920,2.281287,25
1,148,DVN,44.955,1.904802,44
2,175,FANG,114.480,1.585831,17
3,299,MCHP,87.080,1.479823,22
4,195,FTNT,320.530,1.476889,6
5,317,MRO,16.250,1.46531,123
6,345,NVDA,317.900,1.385236,6
7,174,F,19.905,1.175761,100
8,344,NUE,113.240,1.08169,17
9,251,IT,319.490,1.031696,6


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [34]:
hqm_columns=[
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One Year Price Return',
    'One Year Price Percentile',
    'Six Month Price Return',
    'Six Month Price Percentile',
    'Three Month Price Return',
    'Three Month Price Percentile',
    'One Month Price Return',
    'One Month Price Percentile',
    'HQM Score'
    
]

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [59]:
hqm_dataframe = pd.DataFrame(columns=hqm_columns)

for symbol_string in symbol_strings:
    batch_api_url = f"/stock/market/batch?symbols={symbol_string}&types=quote,stats&token={IEX_CLOUD_API_TOKEN}"
    #print(api_url+batch_api_url)
    data = requests.get(api_url+batch_api_url).json()
    
    for symbol in symbol_string.split(','):
        hqm_dataframe=hqm_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    "N/A",
                    data[symbol]['stats']['year1ChangePercent'],
                    "N/A",
                    data[symbol]['stats']['month6ChangePercent'],
                    "N/A",
                    data[symbol]['stats']['month3ChangePercent'],
                    "N/A",
                    data[symbol]['stats']['month1ChangePercent'],
                    "N/A",
                    "N/A"
                ],index=hqm_columns
            ),ignore_index=True
        )
    
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One Year Price Return,One Year Price Percentile,Six Month Price Return,Six Month Price Percentile,Three Month Price Return,Three Month Price Percentile,One Month Price Return,One Month Price Percentile,HQM Score
0,A,155.740,N/A,0.352114,N/A,0.120189,N/A,-0.133516,N/A,-0.016687,N/A,N/A
1,AAL,18.390,N/A,0.039512,N/A,-0.262268,N/A,-0.083973,N/A,-0.18011,N/A,N/A
2,AAP,232.680,N/A,0.548642,N/A,0.22947,N/A,0.181597,N/A,0.022722,N/A,N/A
3,AAPL,174.660,N/A,0.3918,N/A,0.363674,N/A,0.09407,N/A,0.131544,N/A,N/A
4,ABBV,121.420,N/A,0.203121,N/A,0.100382,N/A,0.127975,N/A,0.036952,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,132.355,N/A,0.268538,N/A,0.118261,N/A,0.023214,N/A,0.04058,N/A,N/A
501,ZBH,129.580,N/A,-0.112269,N/A,-0.18415,N/A,-0.122666,N/A,-0.093393,N/A,N/A
502,ZBRA,606.525,N/A,0.61956,N/A,0.196096,N/A,0.041814,N/A,-0.003391,N/A,N/A
503,ZION,63.420,N/A,0.540072,N/A,0.12896,N/A,0.127249,N/A,0.000106,N/A,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [60]:
from statistics import mean
time_periods=['One Month','Three Month','Six Month','One Year']
hqm_dataframe = hqm_dataframe.fillna(value=0)
for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        hqm_dataframe.loc[row,f'{time_period} Price Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row,f'{time_period} Price Return'])
        momentum_percentiles.append(hqm_dataframe.loc[row,f'{time_period} Price Percentile'])
    hqm_dataframe.loc[row,'HQM Score'] = mean(momentum_percentiles)

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One Year Price Return,One Year Price Percentile,Six Month Price Return,Six Month Price Percentile,Three Month Price Return,Three Month Price Percentile,One Month Price Return,One Month Price Percentile,HQM Score
0,A,155.740,N/A,0.352114,65.742574,0.120189,67.722772,-0.133516,7.524752,-0.016687,42.376238,45.841584
1,AAL,18.390,N/A,0.039512,22.178218,-0.262268,3.168317,-0.083973,14.257426,-0.180110,1.386139,10.247525
2,AAP,232.680,N/A,0.548642,85.346535,0.229470,85.148515,0.181597,92.277228,0.022722,70.49505,83.316832
3,AAPL,174.660,N/A,0.391800,69.50495,0.363674,95.445545,0.094070,73.663366,0.131544,98.415842,84.257426
4,ABBV,121.420,N/A,0.203121,43.168317,0.100382,63.564356,0.127975,83.366337,0.036952,79.009901,67.277228
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,132.355,N/A,0.268538,52.673267,0.118261,67.128713,0.023214,51.089109,0.040580,80.990099,62.970297
501,ZBH,129.580,N/A,-0.112269,7.326733,-0.184150,9.50495,-0.122666,8.910891,-0.093393,9.306931,8.762376
502,ZBRA,606.525,N/A,0.619560,90.693069,0.196096,79.80198,0.041814,56.633663,-0.003391,51.089109,69.554455
503,ZION,63.420,N/A,0.540072,84.554455,0.128960,70.693069,0.127249,83.168317,0.000106,54.851485,73.316832


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [61]:
hqm_dataframe.sort_values('HQM Score',ascending = False,inplace=True)
hqm_dataframe=hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace = True)
hqm_dataframe

,index,Ticker,Price,Number of Shares to Buy,One Year Price Return,One Year Price Percentile,Six Month Price Return,Six Month Price Percentile,Three Month Price Return,Three Month Price Percentile,One Month Price Return,One Month Price Percentile,HQM Score
0,345,NVDA,319.540,N/A,1.385236,98.811881,0.841428,99.80198,0.431072,98.811881,0.090043,95.445545,98.217822
1,54,AZO,1981.540,N/A,0.748708,95.445545,0.461431,98.217822,0.325870,97.227723,0.113112,97.425743,97.079208
2,275,LB,79.920,N/A,2.281287,100.0,0.815831,99.60396,0.215421,94.455446,0.079271,93.861386,96.980198
3,243,INTU,673.360,N/A,0.818956,97.029703,0.476224,98.613861,0.202843,93.465347,0.094815,96.039604,96.287129
4,287,LOW,255.055,N/A,0.708519,93.861386,0.360915,95.247525,0.256837,95.841584,0.090728,95.643564,95.148515
5,299,MCHP,87.010,N/A,1.479823,99.405941,1.328878,100.0,1.285190,100.0,0.039407,80.19802,94.90099
6,217,HD,409.180,N/A,0.617706,89.70297,0.355097,94.851485,0.270410,96.237624,0.134368,98.613861,94.851485
7,495,XLNX,225.480,N/A,0.512300,81.782178,0.755490,99.207921,0.448974,99.009901,0.127449,98.019802,94.50495
8,148,DVN,44.875,N/A,1.904802,99.80198,0.476469,98.811881,0.558678,99.80198,0.035337,78.019802,94.108911
9,226,HPQ,36.795,N/A,0.619171,90.49505,0.236787,86.336634,0.310440,97.029703,0.175091,99.405941,93.316832


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [56]:
portfolio_input()

position_size = float(portfolio_size)/len(hqm_dataframe.index)

for i in hqm_dataframe.index:
    hqm_dataframe.loc[i,"Number of Shares to Buy"] = math.floor(position_size/hqm_dataframe.loc[i,"Price"])

hqm_dataframe

Enter the value you want to invest 100000


,level_0,index,Ticker,Price,Number of Shares to Buy,One Year Price Return,One Year Price Percentile,Six Month Price Return,Six Month Price Percentile,Three Month Price Return,Three Month Price Percentile,One Month Price Return,One Month Price Percentile,HQM Score
0,0,0,A,155.820,12,0.352114,65.742574,0.120189,67.722772,-0.133516,7.524752,-0.016687,42.376238,45.841584
1,1,1,AAL,18.390,108,0.039512,22.178218,-0.262268,3.168317,-0.083973,14.257426,-0.180110,1.386139,10.247525
2,2,2,AAP,232.280,8,0.548642,85.346535,0.229470,85.148515,0.181597,92.277228,0.022722,70.49505,83.316832
3,3,3,AAPL,174.440,11,0.391800,69.50495,0.363674,95.445545,0.094070,73.663366,0.131544,98.415842,84.257426
4,4,4,ABBV,121.350,16,0.203121,43.168317,0.100382,63.564356,0.127975,83.366337,0.036952,79.009901,67.277228
5,5,5,ABC,120.560,16,0.219699,45.940594,0.018099,44.752475,-0.034224,27.524752,-0.059661,17.425743,33.910891
6,6,6,ABMD,322.930,6,0.179859,39.80198,0.101769,64.356436,-0.147315,6.336634,-0.113157,6.336634,29.207921
7,7,7,ABT,131.740,15,0.254589,51.287129,0.213067,82.574257,0.035371,54.851485,0.064726,90.693069,69.851485
8,8,8,ACN,372.385,5,0.507942,81.386139,0.323554,93.465347,0.090639,73.069307,0.007816,60.792079,77.178218
9,9,9,ADBE,644.700,3,0.320386,60.594059,0.275757,89.50495,-0.017282,33.267327,-0.019254,41.386139,56.188119


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [64]:
writer = pd.ExcelWriter('recommended_allocations_HQM.xlsx',engine='xlsxwriter')

hqm_dataframe.to_excel(writer, "Recommended Allocations HQM",index = False)
background_color = '#0a0a23'
font_color = '#ffffff'

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [65]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [67]:
column_format = {
    'A':['Ticker',string_template],
    'B':['Ticker',dollar_template],
    'C':['Ticker',dollar_template],
    'D':['Ticker',integer_template],
    'E':['Ticker',integer_template],
    'F':['Ticker',integer_template],
    'G':['Ticker',integer_template],
    'H':['Ticker',integer_template],
    'I':['Ticker',integer_template],
    'J':['Ticker',integer_template],
    'K':['Ticker',integer_template],
    'L':['Ticker',integer_template],
    'M':['Ticker',integer_template],
    'N':['Ticker',integer_template],
}
for column in column_format.keys():
    writer.sheets['Recommended Allocations HQM'].set_column(f"{column}:{column}",18,column_format[column][1])


## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [68]:
writer.save()